In [1]:
import pandas as pd
from stepmix.stepmix import StepMix
from sklearn.metrics import rand_score

In [2]:
# Goal: determine if latent class analysis (LCA) using stepmix can differentiate between abiotic and biotic worlds
# using atmospheric chemical reaction network topological metrics

In [3]:
# import exoplanet data; see Fisher et al 2025 for more info
abiotic_flux=pd.read_csv('Archean Earth flux network metrics, no life.csv') #typical abiotic case
biotic_flux=pd.read_csv('Archean Earth flux network metrics, with life.csv') #biotic case
abiotic_steady_state=pd.read_csv('Archean Earth steady state network metrics, no life.csv') #weird abiotic case

In [4]:
# first, let's analyze the typical abiotic case and the biotic case

In [12]:
exo_combined=pd.concat([abiotic_flux,biotic_flux])
exo_data=exo_combined[['Mean degree','Average shortest path length','CH4 abundance']]

In [13]:
model = StepMix(n_components=2, measurement="categorical",verbose=1)
model.fit(exo_data)

Fitting StepMix...


Initializations (n_init) : 100%|█| 1/1 [00:00<00:00,  9.58it/s, max_LL=-1.58e+4,

MODEL REPORT
    Measurement model parameters
          model_name                            categorical        
          class_no                                        0       1
          param variable                                           
          pis   Average shortest path length_10      0.0132  0.3288
                Average shortest path length_11      0.0717  0.1488
                Average shortest path length_12      0.2362  0.8100
                Average shortest path length_13      0.9998  0.1595
                Average shortest path length_14      0.2606  0.0000
                Average shortest path length_4       0.0000  0.0009
                Average shortest path length_5       0.0000  0.0014
                Average shortest path length_6       0.0000  0.0017
                Average shortest path length_7       0.0000  0.0068
                Average shortest path length_8       0.0020  0.1090
                Average shortest path length_9       0.0090  0.4101
  

StepMix(measurement='categorical', verbose=1)

In [14]:
#calculate predictions based on LCA weights and stats
exo_data['exo_predict']=model.predict(exo_data)

/tmp/ipykernel_14360/1700208285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exo_data['exo_predict']=model.predict(exo_data)


In [15]:
#Now let's see how accurate the predictions turned out to be, first using crosstabs
exo_crosstabs=pd.crosstab(exo_data['exo_predict'],exo_combined['Has life?'])
print(exo_crosstabs)

Has life?       0     1
exo_predict            
0             466  1984
1            4210    41


In [16]:
#And then by calculating the Rand score
rand=rand_score(exo_data['exo_predict'],exo_combined['Has life?'])
print(rand)

0.8601074021030499


In [ ]:
#Not bad! Let's make it a little more interesting by including the weird steady state abiotic case

In [23]:
exo_combined=pd.concat([abiotic_flux,biotic_flux,abiotic_steady_state])
exo_data=exo_combined[['Mean degree','Average shortest path length','CH4 abundance']]

In [24]:
model = StepMix(n_components=2, measurement="categorical",verbose=1)
model.fit(exo_data)

Fitting StepMix...


Initializations (n_init) : 100%|█| 1/1 [00:00<00:00,  6.48it/s, max_LL=-1.99e+4,

MODEL REPORT
    Measurement model parameters
          model_name                            categorical        
          class_no                                        0       1
          param variable                                           
          pis   Average shortest path length_10      0.3543  0.0594
                Average shortest path length_11      0.0470  0.2965
                Average shortest path length_12      0.9220  0.1653
                Average shortest path length_13      0.0651  0.9165
                Average shortest path length_14      0.0000  0.2214
                Average shortest path length_4       0.0000  0.0010
                Average shortest path length_5       0.0000  0.0015
                Average shortest path length_6       0.0000  0.0018
                Average shortest path length_7       0.0054  0.0023
                Average shortest path length_8       0.1190  0.0065
                Average shortest path length_9       0.4676  0.0070
  

StepMix(measurement='categorical', verbose=1)

In [25]:
# Generate predictions
exo_data['exo_predict']=model.predict(exo_data)

/tmp/ipykernel_14360/109608242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exo_data['exo_predict']=model.predict(exo_data)


In [26]:
# Now let's see how we did

exo_crosstabs=pd.crosstab(exo_data['exo_predict'],exo_combined['Has life?'])

print(exo_crosstabs)

rand=rand_score(exo_data['exo_predict'],exo_combined['Has life?'])

print(rand)


Has life?       0     1
exo_predict            
0            3752     3
1            1924  2022
0.6247241292663965


In [28]:
# Hmmmm...not as good. But we know from Fisher et al 2025 that CH4 abundance can actually be confounding when
# comparing the abiotic steady state case and the biotic case. Let's try using a different metric

In [29]:
exo_data=exo_combined[['Mean degree','Average shortest path length','Clustering coefficient']]
model = StepMix(n_components=2, measurement="categorical",verbose=1)
model.fit(exo_data)

Fitting StepMix...


Initializations (n_init) : 100%|█| 1/1 [00:00<00:00,  3.75it/s, max_LL=-1.14e+4,

MODEL REPORT
    Measurement model parameters
          model_name                            categorical        
          class_no                                        0       1
          param variable                                           
          pis   Average shortest path length_10      0.1135  0.0000
                Average shortest path length_11      0.8925  1.0000
                Average shortest path length_12      0.9502  0.0000
                Average shortest path length_13      0.0000  0.9999
                Average shortest path length_9       0.0042  0.0000
                Mean degree_11                       0.0198  0.0000
                Mean degree_12                       0.2096  0.0000
                Mean degree_13                       0.7706  0.2020
                Mean degree_14                       0.0000  0.7980
    Class weights
        Class 1 : 0.56
        Class 2 : 0.44
    Fit for 2 latent classes
    Estimation method             : 1-step
  

StepMix(measurement='categorical', verbose=1)

In [30]:
exo_data['exo_predict']=model.predict(exo_data)

exo_crosstabs=pd.crosstab(exo_data['exo_predict'],exo_combined['Has life?'])

print(exo_crosstabs)

rand=rand_score(exo_data['exo_predict'],exo_combined['Has life?'])

print(rand)

Has life?       0     1
exo_predict            
0            4288     2
1            1388  2023
0.7041271415248821


/tmp/ipykernel_14360/4091565626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exo_data['exo_predict']=model.predict(exo_data)


In [ ]:
# Well, that's a little better
# Next step: investigating relationships between spectral features and network metrics